In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

from auxiliary import values as v
from auxiliary.utils.colors import bcolors as c
from auxiliary.data import imaging
from auxiliary.utils.timer import LoadingBar

from filtering import cardiac_region as cr
from filtering.run_filter_tissue import filter_by_tissue
from feature_extraction.feature_extractor import filter_by_margin, filter_by_volume
from auxiliary.data.dataset_ht import HtDataset
from nuclei_segmentation.run_cellpose import predict
import json


ds = HtDataset()

GPU activated: False


In [3]:
specimen = '0806_E5'
img_path = v.data_path + 'Gr1/RawImages/Nuclei/20190208_E2_DAPI_decon_0.5_crop.nii.gz'
img_path_gt = v.data_path + 'Gr1/Segmentation/Nuclei/20190208_E2_nuclei_mask_crop_GT.nii.gz'

img = imaging.read_image(img_path, axes='XYZ', verbose=1)
img_gt = imaging.read_image(img_path_gt, axes='XYZ', verbose=1)

Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/RawImages/Nuclei/20190208_E2_DAPI_decon_0.5_crop.nii.gz
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/20190208_E2_nuclei_mask_crop_GT.nii.gz


In [43]:
import SimpleITK as sitk
import pandas as pd
# from ctrl.algorithm.image_overlap import fast_image_overlap3d
from auxiliary.utils.timer import LoadingBar

import concurrent.futures


def pixel_accuracy(pred, gt):
    # Ensure binary masks
    pred = (pred > 0).astype(np.uint8)
    gt = (gt > 0).astype(np.uint8)
    
    correct = np.sum(pred == gt)
    total = pred.size
    return correct / total

def iou(pred, gt):
    pred = (pred > 0).astype(np.uint8)
    gt = (gt > 0).astype(np.uint8)
    
    intersection = np.logical_and(pred, gt).sum()
    union = np.logical_or(pred, gt).sum()
    
    if union == 0:
        return 0.0
    
    return intersection / union

def precision(pred, gt):
    pred = (pred > 0).astype(np.uint8)
    gt = (gt > 0).astype(np.uint8)
    
    true_positive = np.logical_and(pred == 1, gt == 1).sum()
    false_positive = np.logical_and(pred == 1, gt == 0).sum()
    
    if true_positive + false_positive == 0:
        return 0.0
    
    return true_positive / (true_positive + false_positive)

def recall(pred, gt):
    pred = (pred > 0).astype(np.uint8)
    gt = (gt > 0).astype(np.uint8)
    
    true_positive = np.logical_and(pred == 1, gt == 1).sum()
    false_negative = np.logical_and(pred == 0, gt == 1).sum()
    
    if true_positive + false_negative == 0:
        return 0.0
    
    return true_positive / (true_positive + false_negative)

def dice_coefficient(pred, gt):
    pred = (pred > 0).astype(np.uint8)
    gt = (gt > 0).astype(np.uint8)
    
    intersection = np.logical_and(pred, gt).sum()
    
    return 2 * intersection / (pred.sum() + gt.sum())


def f1_score(pred, gt):
    p = precision(pred, gt)
    r = recall(pred, gt)
    
    if p + r == 0:
        return 0.0
    
    return 2 * (p * r) / (p + r)

def hausdorff_distance(pred, gt):
    pred = (pred > 0).astype(np.uint8)
    gt = (gt > 0).astype(np.uint8)
    
    pred_image = sitk.GetImageFromArray(pred)
    gt_image = sitk.GetImageFromArray(gt)
    
    hausdorff_filter = sitk.HausdorffDistanceImageFilter()
    hausdorff_filter.Execute(pred_image, gt_image)
    
    return hausdorff_filter.GetHausdorffDistance()

def jacquard_index(pred, gt, return_mean=True):
    """If return mean is True, it returns the mean of the rates of under-segmentation and over-segmentation. If false, return a tuple with the rates of under-segmentation and over-segmentation for each cell."""
    pred_cells = [cell for cell in np.unique(pred) if cell != 0]
    gt_cells = [cell for cell in np.unique(gt) if cell != 0]
    
    # df_jaccard = fast_image_overlap(
    #     mother_seg=pred, daughter_seg=gt,
    #     mother_label=pred_cells, daughter_label=gt_cells,
    #     method='jaccard', ds=1, verbose=False
    # )
    
    jaccard_results = []

    for pred_cell in pred_cells:
        pred_mask = (pred == pred_cell)
        for gt_cell in gt_cells:
            gt_mask = (gt == gt_cell)
            
            # Compute Intersection and Union
            intersection = np.logical_and(pred_mask, gt_mask).sum()
            union = np.logical_or(pred_mask, gt_mask).sum()
            
            jaccard_index = intersection / union if union > 0 else 0
            
            jaccard_results.append({'pred': pred_cell, 'gt': gt_cell, 'jaccard': jaccard_index})
    
    df_jaccard = pd.DataFrame(jaccard_results)
    df_jaccard.columns = ['pred', 'gt', 'jaccard']
    
    # Indentify for each cell, the rarget cell that maximizes their jaccard index
    df_jaccard = df_jaccard.loc[df_jaccard.groupby('pred')['jaccard'].idxmax()]
    
    # Add the missing reference cells anc calculate the volumen of each reference cells
    missing_cells = set(gt_cells) - set(df_jaccard['gt'].values)
    
    rows = []
    if len(missing_cells) > 0:
        rows = [{'pred': 0, 'gt': cell, 'jaccard': 0} for cell in missing_cells]
            
    df_missing = pd.DataFrame(rows)
    df_jaccard = pd.concat([df_jaccard, df_missing], ignore_index=True)
            
    # Add corresponding volumnes (in voxel units)
    unique_gt_cells = np.unique(gt)
    cell_gt_vol = np.array([np.sum(gt == cell) for cell in unique_gt_cells])
    cell_gt_vol_dict = dict(zip(unique_gt_cells, cell_gt_vol))
    
    df_jaccard['volume'] = df_jaccard.apply(lambda x: cell_gt_vol_dict[x['gt']], axis=1)
    
    # Calculate weighted jaccard index by multiplying the jaccard index by the cell volume
    df_jaccard['weighted_jaccard'] = df_jaccard['jaccard'] * df_jaccard['volume']
    
    # The volume averaged jaccard index is obtained by summing all the weighted jaccard index and divide them by the total volume of the ground-truth tissue.
    
    total_cell_volume = sum(df_jaccard['volume'].values)
    sum_weighted_jaccard = sum(df_jaccard['weighted_jaccard'].values)
    vji = sum_weighted_jaccard / total_cell_volume
    
    if return_mean is False:
        return df_jaccard['jaccard'].values
    
    return vji

def segmentation_rates(pred, gt, return_mean=True, compue_maps=False, thr=5):
    """If return mean is True, it returns the mean of the rates of under-segmentation and over-segmentation. If false, return a tuple with the rates of under-segmentation and over-segmentation for each cell."""
    pred_cells = [cell for cell in np.unique(pred) if cell != 0]
    gt_cells = [cell for cell in np.unique(gt) if cell != 0]
    
    # Initialize dictionaries to store the segmented areas
    gt_to_pred = {cell: [] for cell in gt_cells}
    pred_to_gt = {cell: [] for cell in pred_cells}
    
    for pred_cell in pred_cells:
        pred_mask = (pred == pred_cell)
        for gt_cell in gt_cells:
            gt_mask = (gt == gt_cell)
            
            intersection = np.logical_and(pred_mask, gt_mask).sum()
            if intersection > thr:
                gt_to_pred[gt_cell].append(pred_cell)
                pred_to_gt[pred_cell].append(gt_cell)
    
    if return_mean is False:
        rates_under_segmentation = []
        rates_over_segmentation = []
        
        for gt_cell, pred_matches in gt_to_pred.items():
            if len(pred_matches) > 1:
                rates_under_segmentation.append(1)
            else:
                rates_under_segmentation.append(0)
                
        for pred_cell, gt_matches in pred_to_gt.items():
            if len(gt_matches) > 1:
                rates_over_segmentation.append(1)
            else:
                rates_over_segmentation.append(0)
                
        if compue_maps:
            return rates_under_segmentation, rates_over_segmentation, gt_to_pred, pred_to_gt
                
        return rates_under_segmentation, rates_over_segmentation
            
    
    # Calculate Rate of Under-Segmentation (RUS)
    under_segmented_gt = 0
    for gt_cell, pred_matches in gt_to_pred.items():
        if len(pred_matches) > 1:
            under_segmented_gt += 1
    
    # Rate of Under-Segmentation
    RUS = under_segmented_gt / len(gt_cells) if len(gt_cells) > 0 else 0
    
    # Calculate Rate of Over-Segmentation (ROS)
    over_segmented_pred = 0
    for pred_cell, gt_matches in pred_to_gt.items():
        if len(gt_matches) > 1:
            over_segmented_pred += 1
    
    # Rate of Over-Segmentation
    ROS = over_segmented_pred / len(pred_cells) if len(pred_cells) > 0 else 0
    
    return RUS, ROS


# Function to compute all metrics
def compute_metrics(pred, gt):
    print('Computing metrics...')
    
    metrics = {}
    metrics['Pixel Accuracy'] = pixel_accuracy(pred, gt)
    metrics['IoU'] = iou(pred, gt)
    metrics['Precision'] = precision(pred, gt)
    metrics['Recall'] = recall(pred, gt)
    metrics['F1-Score'] = f1_score(pred, gt)
    metrics['Dice Coefficient'] = dice_coefficient(pred, gt)
    metrics['Hausdorff Distance'] = hausdorff_distance(pred, gt)
    metrics['VJI'] = jacquard_index(pred, gt)
    
    return metrics

def compute_metrics_for_cell(gt_cell, pred_cells, pred, gt):
    gt_mask = (gt == gt_cell)
    best_iou = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_dice = 0.0
    best_hausdorff = float('inf')
    
    # Loop through all cells in the prediction
    for pred_cell in pred_cells:
        pred_mask = (pred == pred_cell)
        
        # Compute Jaccard Index (IoU)
        intersection = np.logical_and(pred_mask, gt_mask).sum()
        union = np.logical_or(pred_mask, gt_mask).sum()
        iou = intersection / union if union > 0 else 0.0
        if iou > best_iou:
            best_iou = iou

        # Compute Precision
        true_positive = np.logical_and(pred_mask, gt_mask).sum()
        false_positive = np.logical_and(pred_mask, ~gt_mask).sum()
        precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
        if precision > best_precision:
            best_precision = precision

        # Compute Recall
        false_negative = np.logical_and(~pred_mask, gt_mask).sum()
        recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
        if recall > best_recall:
            best_recall = recall
        
        # Compute Dice Coefficient
        dice = 2 * intersection / (pred_mask.sum() + gt_mask.sum()) if (pred_mask.sum() + gt_mask.sum()) > 0 else 0.0
        if dice > best_dice:
            best_dice = dice
        
        # Compute Hausdorff Distance
        pred_image = sitk.GetImageFromArray(pred_mask.astype(np.uint8))
        gt_image = sitk.GetImageFromArray(gt_mask.astype(np.uint8))
        hausdorff_filter = sitk.HausdorffDistanceImageFilter()
        hausdorff_filter.Execute(pred_image, gt_image)
        hausdorff_distance = hausdorff_filter.GetHausdorffDistance()
        if hausdorff_distance < best_hausdorff:
            best_hausdorff = hausdorff_distance
    
    return best_iou, best_precision, best_recall, best_dice, best_hausdorff


def compute_metrics_cell_aware(pred, gt):
    print('Computing metrics (cell-aware)...')
    
    pred_cells = np.unique(pred)[1:]  # Ignore background (0)
    gt_cells = np.unique(gt)[1:]  # Ignore background (0)
    
    metrics = {
        'Pixel Accuracy': 0.0,
        'Mean IoU': 0.0,
        'Mean Precision': 0.0,
        'Mean Recall': 0.0,
        'Mean Dice Coefficient': 0.0,
        'Mean Hausdorff Distance': 0.0,
    }
    
    iou_values = []
    precision_values = []
    recall_values = []
    dice_values = []
    hausdorff_values = []

    # Parallelize over the gt_cells
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(compute_metrics_for_cell, gt_cell, pred_cells, pred, gt) for gt_cell in gt_cells]
        
        bar = LoadingBar(len(futures))

        for future in concurrent.futures.as_completed(futures):
            best_iou, best_precision, best_recall, best_dice, best_hausdorff = future.result()
            iou_values.append(best_iou)
            precision_values.append(best_precision)
            recall_values.append(best_recall)
            dice_values.append(best_dice)
            hausdorff_values.append(best_hausdorff)
            
            bar.update()
            
        bar.end()

    # Average metrics
    metrics['Mean IoU'] = np.mean(iou_values)
    metrics['Mean Precision'] = np.mean(precision_values)
    metrics['Mean Recall'] = np.mean(recall_values)
    metrics['Mean Dice Coefficient'] = np.mean(dice_values)
    metrics['Mean Hausdorff Distance'] = np.mean(hausdorff_values)
    
    # Pixel Accuracy for the entire image
    pred_flat = pred.flatten()
    gt_flat = gt.flatten()
    correct = np.sum(pred_flat == gt_flat)
    total = pred_flat.size
    metrics['Pixel Accuracy'] = correct / total

    return metrics


# results = []
df = pd.DataFrame(columns=['Method', 'Pixel Accuracy', 'IoU', 'Precision', 'Recall', 'F1-Score', 'Dice Coefficient', 'Hausdorff Distance', 'NO_cells', 'VJI', 'ROS', 'RUS'])

In [8]:
aux = pd.read_csv('../nuclei_segmentation/results.csv')
results = [res for res in aux.to_dict(orient='records')]

4


In [21]:
pred_path = v.data_path + 'Gr1/Segmentation/Nuclei/20190208_E2_nuclei_mask_crop_all.nii.gz'
pred = imaging.read_image(pred_path, axes='XYZ', verbose=1)

metric = compute_metrics(pred, img_gt)
metric_cell_aware = compute_metrics_cell_aware(pred, img_gt)

results.append({
    'Method': '3D_5_6_45_M_EQ_BI_AD',
    'Pixel Accuracy': metric['Pixel Accuracy'],
    'IoU': metric['IoU'],
    'Mean IoU': metric_cell_aware['Mean IoU'],
    'Precision': metric['Precision'],
    'Mean Precision': metric_cell_aware['Mean Precision'],
    'Recall': metric['Recall'],
    'Mean Recall': metric_cell_aware['Mean Recall'],
    'F1-Score': metric['F1-Score'],
    'Dice Coefficient': metric['Dice Coefficient'],
    'Mean Dice Coefficient': metric_cell_aware['Mean Dice Coefficient'],
    'Hausdorff Distance': metric['Hausdorff Distance'],
    'Mean Hausdorff Distance': metric_cell_aware['Mean Hausdorff Distance'],
    'NO_cells': len(np.unique(pred)) - 1,
    'VJI': metric['VJI']
})

print(json.dumps(results[-1], indent=4))

Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/20190208_E2_nuclei_mask_crop_all.nii.gz
Computing metrics...
Computing metrics (cell-aware)...
[==================================================] 100.00%
{
    "Method": "3D_5_6_45_M_EQ_BI_AD",
    "Pixel Accuracy": 0.760974025974026,
    "IoU": 0.47201962176466794,
    "Mean IoU": 0.3121630857076779,
    "Precision": 0.9766727507456706,
    "Mean Precision": 0.5690959032544406,
    "Recall": 0.4774018061146774,
    "Mean Recall": 0.40599819420965927,
    "F1-Score": 0.641322459002017,
    "Dice Coefficient": 0.641322459002017,
    "Mean Dice Coefficient": 0.4224909625114698,
    "Hausdorff Distance": 15.84297951775486,
    "Mean Hausdorff Distance": 10.801477172019158,
    "NO_cells": 117,
    "VJI": 0.3453892056531962
}


In [23]:
df = pd.DataFrame(results)
print(df)
df.to_csv('../nuclei_segmentation/results.csv', index=False)

                  Method  Pixel Accuracy       IoU  Mean IoU  Precision  \
0         2D_5_6_45_M_AD        0.859877  0.696358  0.426008   0.958647   
1         2D_5_6_45_M_BI        0.850231  0.672040  0.449228   0.971335   
2      2D_5_6_45_M_AD_BI        0.859429  0.696451  0.442156   0.954204   
3   2D_5_6_45_M_EQ_AD_BI        0.812435  0.590812  0.362690   0.961754   
4      2D_5_6_45_M_BI_AD        0.853214  0.681690  0.424704   0.958743   
5   2D_5_6_45_M_EQ_BI_AD        0.812409  0.591054  0.372580   0.960657   
6         3D_5_6_45_M_AD        0.852604  0.677626  0.400427   0.969924   
7         3D_5_6_45_M_BI        0.855769  0.683367  0.413654   0.975251   
8      3D_5_6_45_M_AD_BI        0.863604  0.702771  0.400685   0.966207   
9   3D_5_6_45_M_EQ_AD_BI        0.761448  0.473282  0.315027   0.975920   
10     3D_5_6_45_M_BI_AD        0.852971  0.678450  0.400550   0.969868   
11  3D_5_6_45_M_EQ_BI_AD        0.760974  0.472020  0.312163   0.976673   

    Mean Precision    Re

Adding rates of under/over segmentation

In [26]:
tests = [
    '2D_5_6_45_M_AD',
    '2D_5_6_45_M_BI',
    '2D_5_6_45_M_AD_BI',
    '2D_5_6_45_M_EQ_AD_BI',
    '2D_5_6_45_M_BI_AD',
    '2D_5_6_45_M_EQ_BI_AD',
    '3D_5_6_45_M_AD',
    '3D_5_6_45_M_BI',
    '3D_5_6_45_M_AD_BI',
    '3D_5_6_45_M_EQ_AD_BI',
    '3D_5_6_45_M_BI_AD',
    '3D_5_6_45_M_EQ_BI_AD'
]

results = pd.read_csv('../nuclei_segmentation/results.csv')

In [27]:
for test in tests:
    pred_path = v.data_path + f'Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_{test}_all.nii.gz' 
    
    pred = imaging.read_image(pred_path, axes='XYZ', verbose=1)
    RUS, ROS = segmentation_rates(pred, img_gt)
    
    idx = results[results['Method'] == test].index[0]
    results.loc[idx, 'ROS'] = ROS
    results.loc[idx, 'RUS'] = RUS
    
    print(json.dumps(results.loc[idx].to_dict(), indent=4))

Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_2D_5_6_45_M_AD_all.nii.gz
{
    "Method": "2D_5_6_45_M_AD",
    "Pixel Accuracy": 0.8598766233766234,
    "IoU": 0.6963580270799841,
    "Mean IoU": 0.4260081188217122,
    "Precision": 0.9586473083703,
    "Mean Precision": 0.6327861915230099,
    "Recall": 0.7179233307946179,
    "Mean Recall": 0.5949038759920595,
    "F1-Score": 0.8210036041491264,
    "Dice Coefficient": 0.8210036041491263,
    "Mean Dice Coefficient": 0.5429760523668099,
    "Hausdorff Distance": 14.0356688476182,
    "Mean Hausdorff Distance": 10.130523801404417,
    "NO_cells": 181,
    "VJI": 0.369787740594249,
    "ROS": 0.7513812154696132,
    "RUS": 0.6861702127659575
}
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_2D_5_6_45_M_BI_all.nii.gz
{
    

Add VJI, ROS and RUS for each cell with the ROS and RUS maps 

In [44]:
results = pd.read_csv('../nuclei_segmentation/results.csv')

bar = LoadingBar(len(tests))

for test in tests:
    pred_path = v.data_path + f'Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_{test}_all.nii.gz' 
    
    pred = imaging.read_image(pred_path, axes='XYZ', verbose=0)
    VJI = jacquard_index(pred, img_gt, return_mean=False)
    RUS, ROS, RUS_map, ROS_map = segmentation_rates(
        pred, img_gt, return_mean=False, compue_maps=True, thr=5
    )
    
    idx = results[results['Method'] == test].index[0]
    results.loc[idx, 'JI_list'] = ','.join(map(str, VJI))
    results.loc[idx, 'ROS_list'] = ','.join(map(str, ROS))
    results.loc[idx, 'RUS_list'] = ','.join(map(str, RUS))
    results.loc[idx, 'ROS_map'] = json.dumps(ROS_map)
    results.loc[idx, 'RUS_map'] = json.dumps(RUS_map)
    
    bar.update()
    
bar.end()

Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_2D_5_6_45_M_AD_all.nii.gz
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_2D_5_6_45_M_BI_all.nii.gz
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_2D_5_6_45_M_AD_BI_all.nii.gz
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_2D_5_6_45_M_EQ_AD_BI_all.nii.gz
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LAB_MT/LAB/Ignacio/Gr1/Segmentation/Nuclei/QC_CROP/20190208_E2_nuclei_mask_crop_2D_5_6_45_M_BI_AD_all.nii.gz
Reading NIfTI: /run/user/1003/gvfs/smb-share:server=tierra.cnic.es,share=sc/LA

In [45]:
results.to_csv('../nuclei_segmentation/results.csv', index=False)